In [1]:
%pip install h2o

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 177.0 MB 131.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 91.5 MB/s eta 0:00:01
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=0533030546660b1c148809ab4577dcff647b0b881df5f2d4b3bf02c973cf12d2
  Stored in directory: /home/mk7516/.cache/pip/wheels/bf/d7/e0/905f059465fafaf07c9e79ba4c8d642ac646b01ea8c38b07d5
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=4f831c65e1c94dc6bcb6bd66006a00f5a1ba67ae0b30925cbe353bfd64aac98a
  Stored in directory: /home/mk7516/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built h2o future
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-18896479/tmpy2b13rl5
  JVM stdout: /state/partition1/job-18896479/tmpy2b13rl5/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-18896479/tmpy2b13rl5/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_mk7516_sujpiz
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.543 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup('spambase')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(3450, 58)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

19:56:02.385: GBM_lr_annealing_selection_AutoML_1_20220501_185624 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_1_20220501_185624

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0010267718232583357
RMSE: 0.03204328046967626
LogLoss: 0.008066493287128022
Null degrees of freedom: 3449
Residual degrees of freedom: 3445
Null deviance: 4607.583535347977
Residual deviance: 55.658803681183365
AIC: 65.65880368118337
AUC: 0.9999957534991167
AUCPR: 0.9999973162391778
Gini: 0.9999915069982335

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5568719021391693: 


,,0,1,Error,Rate
0,0,1338.0,0.0,0.0,(0.0/1338.0)
1,1,2.0,2110.0,0.0009,(2.0/2112.0)
2,Total,1340.0,2110.0,0.0006,(2.0/3450.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.556872,0.999526,194.0
1,max f2,0.215524,0.999338,201.0
2,max f0point5,0.556872,0.999810,194.0
3,max accuracy,0.556872,0.999420,194.0
4,max precision,0.999980,1.000000,0.0
5,max recall,0.215524,1.000000,201.0
6,max specificity,0.999980,1.000000,0.0
7,max absolute_mcc,0.556872,0.998780,194.0
8,max min_per_class_accuracy,0.556872,0.999053,194.0
9,max mean_per_class_accuracy,0.556872,0.999527,194.0



Gains/Lift Table: Avg response rate: 61.22 %, avg score: 61.10 %


...


Gains/Lift Table: Avg response rate: 61.22 %, avg score: 61.19 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010145,0.999985,1.633523,1.633523,1.000000,0.999990,1.000000,0.999990,0.016572,0.016572,63.352273,63.352273,0.016572
1,2,0.020000,0.999979,1.633523,1.633523,1.000000,0.999982,1.000000,0.999986,0.016098,0.032670,63.352273,63.352273,0.032670
2,3,0.030145,0.999973,1.633523,1.633523,1.000000,0.999976,1.000000,0.999983,0.016572,0.049242,63.352273,63.352273,0.049242
3,4,0.040000,0.999966,1.633523,1.633523,1.000000,0.999970,1.000000,0.999979,0.016098,0.065341,63.352273,63.352273,0.065341
4,5,0.050435,0.999958,1.633523,1.633523,1.000000,0.999962,1.000000,0.999976,0.017045,0.082386,63.352273,63.352273,0.082386
5,6,0.100000,0.999918,1.633523,1.633523,1.000000,0.999937,1.000000,0.999957,0.080966,0.163352,63.352273,63.352273,0.163352
6,7,0.150145,0.999867,1.633523,1.633523,1.000000,0.999894,1.000000,0.999936,0.081913,0.245265,63.352273,63.352273,0.245265
7,8,0.200000,0.999810,1.633523,1.633523,1.000000,0.999841,1.000000,0.999912,0.081439,0.326705,63.352273,63.352273,0.326705
8,9,0.300000,0.999551,1.633523,1.633523,1.000000,0.999697,1.000000,0.999840,0.163352,0.490057,63.352273,63.352273,0.490057
9,10,0.400000,0.998631,1.633523,1.633523,1.000000,0.999174,1.000000,0.999674,0.163352,0.653409,63.352273,63.352273,0.653409




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.983597,0.002778,0.983025,0.980282,0.985163,0.982074,0.987441
1,auc,0.998234,0.000822,0.998281,0.997407,0.998893,0.997406,0.999181
2,err,0.016403,0.002778,0.016975,0.019718,0.014837,0.017926,0.012559
3,err_count,11.400000,2.607681,11.000000,14.000000,10.000000,14.000000,8.000000
4,f0point5,0.985855,0.003197,0.982587,0.983834,0.990772,0.985075,0.987007
5,f1,0.986609,0.002406,0.986267,0.983834,0.987893,0.985075,0.989975
6,f2,0.987375,0.003914,0.989975,0.983834,0.985031,0.985075,0.992961
7,lift_top_group,1.632348,0.022338,1.628141,1.639723,1.624096,1.665245,1.604534
8,logloss,0.050178,0.008653,0.051249,0.059575,0.044197,0.056996,0.038874
9,max_per_class_error,0.024315,0.005464,0.032000,0.025271,0.016867,0.022436,0.025000



See the whole table with table.as_data_frame()


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_4_AutoML_1_20220501_185624,0.99822,0.0506008,0.998941,0.0195084,0.118858,0.0141272
GBM_grid_1_AutoML_1_20220501_185624_model_23,0.997941,0.0558914,0.998755,0.0211656,0.124933,0.0156084
GBM_grid_1_AutoML_1_20220501_185624_model_27,0.997743,0.0548339,0.99866,0.0200817,0.123172,0.0151713
GBM_grid_1_AutoML_1_20220501_185624_model_98,0.99774,0.0566662,0.998581,0.0228717,0.125209,0.0156774
GBM_grid_1_AutoML_1_20220501_185624_model_100,0.997733,0.0587334,0.998601,0.0215765,0.126645,0.0160389
GBM_grid_1_AutoML_1_20220501_185624_model_111,0.997711,0.0564435,0.99863,0.0203928,0.123634,0.0152854
GBM_grid_1_AutoML_1_20220501_185624_model_15,0.997704,0.0562342,0.99866,0.0199076,0.12448,0.0154954
GBM_grid_1_AutoML_1_20220501_185624_model_62,0.997692,0.0573879,0.998589,0.0207431,0.126324,0.0159577
GBM_grid_1_AutoML_1_20220501_185624_model_19,0.99764,0.0591794,0.998538,0.0178024,0.127203,0.0161806
GBM_grid_1_AutoML_1_20220501_185624_model_108,0.99757,0.0578996,0.998599,0.0206178,0.125461,0.0157405
